In [8]:
import pandas as pd
import os
import json
import csv
import re, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta

from lifelines import KaplanMeierFitter

from global_functions import *

In [9]:
relase_creation_ts_all = get_release_dates()
all_releases = ["3.0",
    "3.1",
    "3.2", #Callisto
    "3.3", #Europa
    "3.4", #Ganymede
    "3.5", #Galileo
    "3.6", #Helios
    "3.7", #Indigo
    "4.2", #Juno

    "4.3", #Kepler
    "4.4", #Luna
    "4.5", #Mars
    "4.6", #Neon
    "4.7", #Oxygen
    "4.8", #Photon
   '4.9',
    '4.10'
    ]

# Take into account if the bug is resolved in more than one release

In [11]:
def if_minor(version1,version2):
    version1 = str(version1)
    version2 = str(version2)
    if ('.' in  version1) & ('.' in version2):
        parts = version1.split('.')
        release1=parts[0]+'.'+parts[1]
        parts = version2.split('.')
        release2=parts[0]+'.'+parts[1]
        if release1==release2:
            return 0
        return 1
    return 0
    

In [12]:
#load data
df_info=pd.read_csv('data'+os.sep+'bugs_info.zip',index_col=False,compression='zip',
                     dtype={'release':str,'version':str})
df_history=pd.read_csv('data'+os.sep+'bug_history.zip',index_col=False,compression='zip')
#df_all = df_all.fillna('')
df_info = trans_to_datetime(df_info,['creation_time',
                             'first_resolved_date','last_resolved_date',
                             'first_fixed_date','last_fixed_date',
                             'first_assignment_date','last_assignment_date'])
df_history = trans_to_datetime(df_history,['when'])
#df_info.head()

In [13]:
#should be resolved
df_info=df_info[(~df_info.last_resolved_date.isnull()) & (~df_info.first_resolved_date.isnull())]

In [14]:
#resolved in more than one one time
df_info=df_info[df_info['first_resolved_date']!=df_info['last_resolved_date']]

In [17]:
df_history1=df_history.copy()
df_history1=df_history1[(df_history1['what']=='version') ]

In [18]:
#remove the row where the change in version is from unspecified or invalid version
df_history1=df_history1[(df_history1['what']=='version') & (df_history1['removed']!='unspecified') & (df_history1['added']!='unspecified')]

df_history1=df_history1[ (df_history1['added']>='3.0') & (df_history1['removed']>='3.0') & (df_history1['added']<='4.10') & (df_history1['removed']<='4.10')]
df_history1['keep'] = df_history1.apply(lambda x: if_minor(str(x['added']),str(x['removed'])),axis=1)
df_history1 =df_history1[df_history1['keep']==1]

In [20]:
#get the history of the bugs that fulfil the previous conditions
df_history=df_history[df_history.id.isin(df_history1.id.unique())]
df_history=df_history[df_history.id.isin(df_info.id.unique())]

In [22]:
df_history=df_history[(df_history['what']=='version') | (df_history['what']=='resolution') ]

In [24]:
#again remove the unspecified version
df_history=df_history[((df_history['what']=='version') & 
                       ((df_history['removed']!='unspecified') & (df_history['added']!='unspecified'))
                      )  | (df_history['what']=='resolution') ]


In [25]:
#assign th minor bugs to the major ones
df_history['removed'] = df_history.apply(lambda x : str(assign_to_closest_minor(x['removed'])) if x['what']=='version' else x['removed'], axis=1)
df_history['added'] = df_history.apply(lambda x : str(assign_to_closest_minor(x['added'])) if x['what']=='version' else x['added'], axis=1)

In [26]:
df_history=df_history[((df_history['what']=='version') & 
                       ((df_history['removed'].isin(all_releases) & (df_history['added'].isin(all_releases)))
                      )) | (df_history['what']=='resolution') ]
df_history[df_history['added'].notnull()]
df_history=df_history.reset_index()
df_history.loc[0, 'does'] = 0


,added,id,removed,what,when,who
924,3.1,100273,3.0,version,2005-06-15 19:06:51,mikhailkhod@googlemail.com
926,LATER,100273,NaN,resolution,2005-06-17 12:59:37,darin.eclipse@gmail.com
935,FIXED,100273,NaN,resolution,2005-11-18 18:47:09,chanskw@ca.ibm.com
4134,FIXED,101152,NaN,resolution,2005-06-22 14:54:35,Tod_Creasey@ca.ibm.com
4138,3.2,101152,3.1,version,2006-01-24 22:47:19,rcernich@sybase.com
4140,FIXED,101152,NaN,resolution,2006-01-25 13:27:05,Tod_Creasey@ca.ibm.com
4270,LATER,101224,NaN,resolution,2005-06-22 13:57:49,Darin_Swanson@us.ibm.com
4279,3.5,101224,3.1,version,2009-06-25 07:25:25,mhilpert@gmx.de
4285,WORKSFORME,101224,NaN,resolution,2011-06-06 20:59:50,Michael_Rennie@ca.ibm.com
4905,INVALID,101453,NaN,resolution,2005-06-23 14:44:30,dj.houghton@ca.ibm.com


In [32]:
for i in range(0, len(df_history)-1):
   # print(df_history.loc[i, 'what'])
    if df_history.loc[i, 'what']=='version':
        if (df_history.loc[i+1, 'what']=='resolution') &  (df_history.loc[i+1, 'id']==df_history.loc[i, 'id']):# & (df_history.loc[i, 'added']!=str(df_info[df_info['id']==df_history.loc[i, 'id']].release.unique()[0])) :
            #print(str(df_info[df_info['id']==df_history.loc[i, 'id']].release.unique()[0]),df_history.loc[i, 'added'] )
            df_history.loc[i, 'does'] = True #+ df.loc[i, 'B']
        else:
            df_history.loc[i, 'does'] = False 
    else:
         df_history.loc[i, 'does'] = False

In [36]:
print('# of bugs resolved in multiple versions:',len(df_history[df_history['does']==True].id.unique()))

# of bugs resolved in multiple versions: 190
